In [7]:
import pandas as pd
import os

# RAIS

## Vínculos


In [8]:
def read_data(caminho, ano, uf)-> pd.DataFrame:
    '''Read the ESTB data from a file and return a DataFrame.'''
    colunas = ['CNAE 2.0 Classe', 'Município']
    df_vinc = pd.read_csv(f'{caminho}{ano}/{uf}', sep=';', encoding='latin1', usecols=colunas)  # voltar para cá a cada UF
    df_vinc['ano'] = ano

    return df_vinc, 'txt'

In [9]:
def transform_data(df, extensao) -> pd.DataFrame:
    '''Normaliza o tipo de cada coluna e obtem a cnae_secao de cnae_2'''
    
    if extensao == 'txt':
        df.rename(columns={'Município': 'id_municipio', 'CNAE 2.0 Classe': 'cnae_2'}, inplace=True)	
        df = df[['ano', 'id_municipio', 'cnae_2']]
        df.dropna(inplace=True)
    elif extensao == 'csv':
        df.dropna(inplace=True)
        df['id_municipio'] = df['id_municipio'].astype('int')
        df['id_municipio'] = df['id_municipio'].apply(lambda x: int(x/10)) # remove 7'th dígito
    
    # para todos
    df['ano'] = df['ano'].astype('int16')
    
    df['id_municipio'] = df['id_municipio'].astype('int')
    
    df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))
    df['cnae_secao'] = df['cnae_2'].apply(lambda x: x[:2]) # 2 primeiros dígitos -> seção
    df['cnae_secao'] = df['cnae_secao'].astype('int')
    df.drop(columns=['cnae_2'], inplace=True)
    return df

In [10]:
def merge_municipios(df, caminho, loc) -> pd.DataFrame:
    '''Faz o merge do id_municipio com os dados do dicionário'''
    
    if loc == 'meso':
        colunas_muni = ['id_municipio_6', 'sigla_uf', 'id_mesorregiao', 'nome_mesorregiao']
    elif loc == 'micro':
        colunas_muni = ['id_municipio_6', 'sigla_uf', 'id_microrregiao', 'nome_microrregiao', 'id_mesorregiao']
    elif loc == 'muni':
        colunas_muni = ['id_municipio_6', 'sigla_uf', 'centroide', 'nome', 'id_microrregiao']
    dicionario_muni = pd.read_csv(caminho, sep=',', usecols=colunas_muni)
    dicionario_muni.rename(columns={'id_municipio_6': 'id_municipio', 'nome': 'municipio'}, inplace=True)
    
    df = pd.merge(df, dicionario_muni, on='id_municipio', how='left')
    return df

In [11]:
def merge_cnae(df, caminho) -> pd.DataFrame:
    
    colunas_tmp = ['divisao','descricao_secao']
    df_cnae = pd.read_csv(caminho, usecols=colunas_tmp)
    df_cnae.rename(columns={'divisao': 'cnae_secao'}, inplace=True)
    df_cnae.drop_duplicates(inplace=True)   
    
    df = pd.merge(df, df_cnae, on='cnae_secao', how='left')
    df.drop(columns=['cnae_secao'], inplace=True)
    return df

In [12]:
def calc_ql(df, loc='meso') -> pd.DataFrame:
    
    if loc == 'meso':
        numerador = df.groupby(['ano', 'sigla_uf', 'id_mesorregiao', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'id_mesorregiao']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql = ql.reset_index()
        ql.columns = ['ano', 'sigla_uf', 'id_mesorregiao', 'descricao_secao', 'quociente_localizacao']
        
    elif loc == 'micro':
        numerador = df.groupby(['ano', 'sigla_uf', 'id_microrregiao', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'id_microrregiao']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql = ql.reset_index()
        ql.columns = ['ano', 'sigla_uf', 'id_microrregiao', 'descricao_secao', 'quociente_localizacao']
        
    elif loc == 'muni':
        numerador = df.groupby(['ano', 'sigla_uf', 'id_municipio', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'id_municipio']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql = ql.reset_index()
        ql.columns = ['ano', 'sigla_uf', 'id_municipio', 'descricao_secao', 'quociente_localizacao']
        
    ql['quociente_localizacao'] = ql['quociente_localizacao'].map(lambda x: float(f"{x:.2f}"))

    return ql

In [13]:
def etl_pipeline(caminhos, ano_ini, ano_fim, loc):

    dfs_ql = []
    for ano in range(ano_ini, ano_fim+1):
        ufs = os.listdir(os.path.join(caminhos['rais'], str(ano)))
        for arq_uf in ufs:
            df, extensao = read_data(caminhos['rais'], str(ano), arq_uf)
            df = transform_data(df, extensao)
            df = merge_municipios(df, caminhos['municipios'], loc)
            df = merge_cnae(df, caminhos['cnae'])
            dfs_ql.append(calc_ql(df, loc))
            
    df = pd.concat(dfs_ql)
    return df

---

In [14]:
from joblib import Parallel, delayed
from pathlib import Path

def processar_loc(loc):
    
    df_ql = etl_pipeline(caminhos, ano_ini, ano_fim, loc)
    path_out = Path(f"C:/dev/ndti/rais/indices/vinculos/ref_estadual/indice_qe_secao_{loc}.csv")
    path_out.parent.mkdir(exist_ok=True)
    df_ql.to_csv(path_out, sep=';', index=False)
    print(f'--- {loc} concluído ---')
    return loc

# Parâmetros globais
caminho = "D:/dados/rais/vinculos/extraidos/"
caminho_dict_muni = "D:/dados/rais/dicionarios/dicionario_municipios.csv"
caminho_dict_cnae = "D:/dados/rais/dicionarios/dicionario_cnae_2.csv"
ano_ini = 2007
ano_fim = 2024
caminhos = {'rais': caminho, 
            'municipios': caminho_dict_muni, 
            'cnae': caminho_dict_cnae}

resultados = Parallel(n_jobs=3)(delayed(processar_loc)(loc) for loc in ['meso', 'micro', 'muni'])